In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
import datetime
import random
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, 'L' code_mode_exercice  from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [144]:
def get_pa(year, specialite):
    sql = f"""
select pa.id, pa.inpp, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
--and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id, ps.code_mode_exercice
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [4]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [5]:
def get_pop_iris(year):
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

In [6]:
def get_iris_matrix(time: int, time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m



In [233]:
specialite=10
year=23
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, specialite, source, time, time_type, accessibilite_exp)
debug=60230000
writer = pd.ExcelWriter(f"out/debug_{debug}_{year}_{specialite}_{time}.xlsx")

23 10 PA 30 HC -0.12


In [8]:
# Very long
iris_matrix = get_iris_matrix(time, time_type)
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= 30) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,10010000,10010000,0,0,0,0,10010000
1,10010000,10050000,21,24,24,24,10050000
2,10010000,10210000,25,30,30,30,10210000
3,10010000,10250000,22,28,30,28,10250000
4,10010000,10260000,20,25,27,25,10260000
...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000
19272671,2023650000,2022510000,10,15,15,15,2022510000
19272672,2023650000,2022770000,16,28,28,28,2022770000
19272673,2023650000,2023650000,0,0,0,0,2023650000


In [9]:
pop_iris = get_pop_iris(year)
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,821300000,821300000,82130,C,170.000000,1.024096,3.072289,8.192771,5.120482,4.096386,28.674699,33.795181,26.626506,44.036145,15.361446
1,811540000,811540000,81154,C,80.000000,2.105263,3.157895,5.263158,4.210526,2.105263,4.210526,17.894737,17.894737,16.842105,6.315789
2,840060000,840060000,84006,C,71.000000,2.028571,3.042857,9.128571,3.042857,2.028571,16.228571,12.171429,9.128571,6.085714,8.114286
3,821570000,821570000,82157,C,107.000000,4.734513,3.787611,4.734513,5.681416,7.575221,16.097345,18.938053,17.991150,22.725664,4.734513
4,840070113,840070113,84007,A,2744.168077,66.531592,76.166839,91.027356,186.769463,466.091795,711.988855,459.891850,281.679718,253.076015,150.944594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48550,940460103,940460103,94046,A,1773.227226,43.209398,54.214737,66.914813,90.574673,141.873872,412.719594,339.034379,197.545292,327.448839,99.691628
48551,940460109,940460109,94046,A,2474.409868,60.984598,64.709729,183.003802,264.548488,197.623284,425.604137,527.371166,305.141054,330.464804,114.958805
48552,940460202,940460202,94046,A,2435.011871,90.000439,77.000375,125.000609,198.000965,427.002082,680.003315,670.003266,145.000707,17.000083,6.000029
48553,940540105,940540105,94054,A,2075.664303,59.233979,69.397063,104.184780,218.764070,149.651718,291.132829,414.269086,338.977538,337.581534,92.471705


In [234]:
pop_iris[pop_iris["iris"]==debug].to_excel(writer, sheet_name="iris", index=None)
pop_iris[pop_iris["iris"]==debug]


,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
34236,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744


In [235]:
ps_df = get_by_source(year, specialite, source)
ps_df

Quering PA for year 23 and specialite 10

select pa.id, pa.inpp, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
--and ps.code_mode_exercice='L'
and ds.annee=23
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id, ps.code_mode_exercice



,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice
0,1905,810000040104,BORDIER,AURORE,17,1292870,172990101,-0.963639,45.936308,S
1,1952,810000225218,FIORENTINO,SIMONE,75,1167890,751114209,2.367627,48.861866,S
2,1952,810000225218,FIORENTINO,SIMONE,75,1288379,751124610,2.393827,48.838107,S
3,1953,810000228659,RICHARD,JEAN LUC,75,45973,751082902,2.304280,48.865722,L
4,1953,810000228659,RICHARD,JEAN LUC,75,45973,751082902,2.304280,48.865722,S
...,...,...,...,...,...,...,...,...,...,...
122019,2537815,810108738757,MAHFOUDHI GUETARI,NESRINE,77,759454,773330104,2.704117,48.261985,S
122020,2538306,810102055711,LOBRY,PAULINE,94,1034686,940670101,2.426930,48.845498,S
122021,2539622,810108699249,HAOUCHE,NACIMA,76,161930,762590104,0.386736,49.745809,S
122022,2540179,810101910890,DUPONT DE DINECHIN,CONSTANCE,94,1034686,940670101,2.426930,48.845498,S


In [236]:
ps_df[ps_df["iris"]==debug].to_excel(writer, sheet_name="ps", index=None)
ps_df[ps_df["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice
37435,377059,810003900114,DI VINCENZO,DOMINIQUE,6,1245439,60230000,7.514719,43.94556,L
37436,377059,810003900114,DI VINCENZO,DOMINIQUE,6,1245439,60230000,7.514719,43.94556,S
72478,794726,810101744653,GIRAUD,HELENE,6,1245439,60230000,7.514719,43.94556,L
77449,859914,810101391927,DUHAMEL,BERTRAND,6,3080,60230000,7.514719,43.94556,L
113128,1398844,810001061463,GERSCHTEIN,JEAN LOUIS,6,1245439,60230000,7.514719,43.94556,L
113129,1398844,810001061463,GERSCHTEIN,JEAN LOUIS,6,1245439,60230000,7.514719,43.94556,S


In [237]:
nb_ps = ps_df["id"].nunique()
nb_cabinet_ps = ps_df.groupby(["id", "lon", "lat"])
print(f"Nb unique PS {nb_ps}")
print(f"Nb cabinet {len(nb_cabinet_ps)}")

Nb unique PS 89544
Nb cabinet 108735


In [238]:
ps_df["key"] = ps_df["id"].astype(str) + "_" + ps_df["code_mode_exercice"] + "_" + ps_df["lat"].astype(str) + "_" + ps_df["lon"].astype(str)
ps_df["nb_cabinet"] = ps_df.groupby("id")["key"].transform("nunique")
ps_df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1     0.615576
4     0.152790
2     0.116551
6     0.051351
3     0.028445
8     0.016046
5     0.005384
10    0.004196
7     0.003270
14    0.002409
12    0.001672
9     0.000959
16    0.000656
28    0.000229
11    0.000180
20    0.000164
15    0.000123
Name: proportion, dtype: float64

In [239]:
ps_df[ps_df["iris"]==debug].to_excel(writer, sheet_name="nb_cabinet", index=None)
ps_df[ps_df["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet
37435,377059,810003900114,DI VINCENZO,DOMINIQUE,6,1245439,60230000,7.514719,43.94556,L,377059_L_43.94556_7.514719,12
37436,377059,810003900114,DI VINCENZO,DOMINIQUE,6,1245439,60230000,7.514719,43.94556,S,377059_S_43.94556_7.514719,12
72478,794726,810101744653,GIRAUD,HELENE,6,1245439,60230000,7.514719,43.94556,L,794726_L_43.94556_7.514719,1
77449,859914,810101391927,DUHAMEL,BERTRAND,6,3080,60230000,7.514719,43.94556,L,859914_L_43.94556_7.514719,2
113128,1398844,810001061463,GERSCHTEIN,JEAN LOUIS,6,1245439,60230000,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6
113129,1398844,810001061463,GERSCHTEIN,JEAN LOUIS,6,1245439,60230000,7.514719,43.94556,S,1398844_S_43.94556_7.514719,6


In [240]:
ps_df = ps_df[ps_df["code_mode_exercice"]=="L"]
ps_df["weight"] = 1 / ps_df["nb_cabinet"]
ps_df["nb"] = ps_df.groupby("iris")["weight"].transform("sum")
ps_df.head(10)
# 0.5 2.17
# 0.5 8.83

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
3,1953,810000228659,RICHARD,JEAN LUC,75,45973,751082902,2.304280,48.865722,L,1953_L_48.865722_2.30428,4,0.250000,1.116667
5,1953,810000228659,RICHARD,JEAN LUC,75,1252865,751156023,2.273756,48.839637,L,1953_L_48.839637_2.273756,4,0.250000,4.416667
7,2018,810000399674,BENNAIM,LAURENT,75,46258,751135206,2.342763,48.838015,L,2018_L_48.838015_2.342763,8,0.125000,95.841667
9,2018,810000399674,BENNAIM,LAURENT,75,421806,751083004,2.309872,48.871651,L,2018_L_48.871651_2.309872,8,0.125000,9.208333
11,2018,810000399674,BENNAIM,LAURENT,75,1157011,751052004,2.346279,48.850155,L,2018_L_48.850155_2.346279,8,0.125000,4.791667
13,2018,810000399674,BENNAIM,LAURENT,75,1292327,751083108,2.322754,48.875020,L,2018_L_48.87502_2.322754,8,0.125000,7.241667
15,2046,810000480375,FAUGERON,DOMINIQUE,91,173812,916870101,2.376338,48.669972,L,2046_L_48.66997244954608_2.3763383823879556,6,0.166667,3.791667
17,2046,810000480375,FAUGERON,DOMINIQUE,92,267700,920020403,2.298845,48.753096,L,2046_L_48.753096_2.298845,6,0.166667,1.166667
19,2046,810000480375,FAUGERON,DOMINIQUE,92,1264517,920330108,2.170714,48.838378,L,2046_L_48.838378_2.170714,6,0.166667,2.583333
21,2054,810000508670,DUPAIN,PIERRE,85,77160,852380000,-0.895764,46.952545,L,2054_L_46.952545_-0.895764,1,1.000000,2.000000


In [241]:
ps_df[ps_df["iris"]==debug].to_excel(writer, sheet_name="nb", index=None)
ps_df[ps_df["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
37435,377059,810003900114,DI VINCENZO,DOMINIQUE,6,1245439,60230000,7.514719,43.94556,L,377059_L_43.94556_7.514719,12,0.083333,1.75
72478,794726,810101744653,GIRAUD,HELENE,6,1245439,60230000,7.514719,43.94556,L,794726_L_43.94556_7.514719,1,1.000000,1.75
77449,859914,810101391927,DUHAMEL,BERTRAND,6,3080,60230000,7.514719,43.94556,L,859914_L_43.94556_7.514719,2,0.500000,1.75
113128,1398844,810001061463,GERSCHTEIN,JEAN LOUIS,6,1245439,60230000,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6,0.166667,1.75


In [242]:
ps_df = ps_df.sort_values(by='iris')
ps_df2 = ps_df.drop_duplicates(subset=['iris', 'nb'])
ps_df2


,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
106894,1294762,810003116513,PAGET,PHILIPPE,1,125,10040102,5.343799,45.954845,L,1294762_L_45.95484508921275_5.343798989531327,1,1.00,2.000000
114221,1416059,810004087242,SCHODTS,PIETER JAN,1,88441,10040201,5.356510,45.961711,L,1416059_L_45.961711_5.35651,1,1.00,7.250000
27731,274008,810100211944,GENOT,PIERRE,1,1179369,10040202,5.352703,45.981523,L,274008_L_45.981523_5.352703,1,1.00,8.583333
94376,1103296,810002950946,PIGNARD,DOMINIQUE,1,79597,10050000,4.902674,45.998332,L,1103296_L_45.998332_4.902674,1,1.00,2.000000
57104,604066,810002950342,BLANCHON,JEAN FRANCOIS,1,88442,10070000,5.358002,46.005238,L,604066_L_46.005238_5.358002,1,1.00,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97206,1143129,810003449682,LEPARC,DOMINIQUE,202,1578899,2023430000,9.453649,42.484316,L,1143129_L_42.484316_9.453649,1,1.00,1.000000
119593,1507682,810003448569,RAFFIN,MICHEL,202,1264929,2023460000,9.439181,42.492594,L,1507682_L_42.492594_9.439181,1,1.00,6.000000
98411,1161071,810100729713,LEQUEUX,JUDY,202,183988,2023530000,9.454906,42.712932,L,1161071_L_42.712932_9.454906,2,0.50,4.000000
12241,113999,810101018306,GHIONGA,PASCAL,202,120664,2023540000,9.169638,42.173293,L,113999_L_42.17329312388466_9.169637721169428,4,0.25,1.250000


In [243]:
ps_df2[ps_df2["iris"]==debug].to_excel(writer, sheet_name="drop", index=None)
ps_df2[ps_df2["iris"]==debug]

,id,inpp,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
113128,1398844,810001061463,GERSCHTEIN,JEAN LOUIS,6,1245439,60230000,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6,0.166667,1.75


In [244]:
iris_matrix_pop_df = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', '')) #iris = iris2
iris_matrix_pop_df

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.0,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.0,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.0,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.0,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000,2B1350000,2B135,C,324.0,3.980971,2.049514,2.049514,2.020847,18.774109,42.392738,42.077099,55.572529,98.952018,56.130660
19272671,2023650000,2022510000,10,15,15,15,2022510000,2B2510000,2B251,C,3737.0,107.555612,118.720070,241.423529,317.512086,210.704067,631.501489,774.648158,501.913516,523.962099,309.059373
19272672,2023650000,2022770000,16,28,28,28,2022770000,2B2770000,2B277,C,333.0,4.577269,2.653059,12.288707,23.776840,23.014588,32.395245,76.114846,64.660108,67.979158,25.540181
19272673,2023650000,2023650000,0,0,0,0,2023650000,2B3650000,2B365,C,100.0,0.000000,0.943396,0.000000,3.773585,9.433962,9.433962,9.433962,23.584906,29.245283,14.150943


In [245]:
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug].to_excel(writer, sheet_name="iris_matrix", index=None)
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
275796,60230000,60230000,0,0,0,0,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744
301290,60620000,60230000,9,11,11,11,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744
369018,61320000,60230000,8,12,12,12,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744
380294,61620000,60230000,19,24,24,24,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744
380299,61630000,60230000,20,26,26,26,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744


In [246]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)
iris_matrix_pop_df["accessibilite_weight"] = accessibilite_fn(iris_matrix_pop_df["time"])
iris_matrix_pop_df.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.000000,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.000000,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108,0.056135
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.000000,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055,0.027324
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.000000,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300,0.034735
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.000000,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472,0.049787
5,10010000,10280000,8,9,9,9,10280000,010280000,01028,C,621.000000,22.723364,29.574898,53.956390,70.586718,22.773615,107.898596,115.308426,89.365644,83.584630,25.227720,0.339596
6,10010000,10420000,14,16,16,16,10420000,010420000,01042,C,290.000000,3.133037,7.312245,22.902721,37.112788,10.199399,41.375370,72.152604,45.323311,42.956047,7.532476,0.146607
7,10010000,10460000,12,13,13,13,10460000,010460000,01046,C,904.000000,36.261938,27.238169,71.537021,113.956388,45.490399,158.230805,197.653434,122.489017,101.564554,29.578275,0.210136
8,10010000,10520000,17,21,22,21,10520000,010520000,01052,C,327.000000,8.933497,12.331319,22.701646,25.430244,8.343897,43.396597,63.486997,59.830525,57.972826,24.572452,0.080460
9,10010000,10530102,26,28,34,28,10530102,010530102,01053,A,1925.526865,47.250779,43.545566,63.504863,104.972858,183.815536,325.024244,323.908369,162.260767,385.334404,285.909479,0.034735


In [247]:
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug].to_excel(writer, sheet_name="accessibilite", index=None)
iris_matrix_pop_df[iris_matrix_pop_df["iris2"]==debug]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
275796,60230000,60230000,0,0,0,0,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744,1.000000
301290,60620000,60230000,9,11,11,11,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744,0.267135
369018,61320000,60230000,8,12,12,12,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744,0.236928
380294,61620000,60230000,19,24,24,24,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744,0.056135
380299,61630000,60230000,20,26,26,26,60230000,060230000,06023,C,2206.0,42.204374,60.148179,98.770346,155.655341,116.810202,317.19745,402.659879,361.104548,465.847938,185.601744,0.044157


In [248]:
cols = [col for col in iris_matrix_pop_df.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [249]:
yy=min(year, 24)
sql = f"""
select o.* from apl.overrepresentation o
join specialite s on s.psp_spe_snds=o.psp_spe_snds
where o.year={yy}
and s.id={specialite}
"""
over = pd.read_sql(sql, config.connection_string)
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,23,1,0.976582,0.451643,0.336768,0.352023,0.640443,0.811534,0.874099,1.108385,1.624337,2.63959


In [250]:
over.to_excel(writer, sheet_name="over", index=None)

In [251]:
if len(over) > 0:
    weights = over.values[0,2:]
else:
    weights = np.ones(len(over.columns) - 2)
weights

array([0.9765819 , 0.45164328, 0.33676791, 0.35202288, 0.64044287,
       0.81153415, 0.87409873, 1.1083846 , 1.6243372 , 2.63958955])

In [252]:
iris_matrix_pop_df["pop_gp"] = sum(w * iris_matrix_pop_df[c] for w, c in zip(weights, cols))
if specialite == 5:
    iris_matrix_pop_df["pop_gp"] /= 2
iris_matrix_pop_df=iris_matrix_pop_df.sort_values(by='iris2')
iris_matrix_pop_df.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000,828.207499
6351493,712700102,10010000,25,27,33,27,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.039164,828.207499
6351706,712700103,10010000,24,26,32,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,828.207499
60021,14430000,10010000,21,26,27,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,828.207499
6351935,712700104,10010000,25,28,34,28,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.034735,828.207499


In [253]:
ratio_mean=1
if specialite!=5:
    test_pop=iris_matrix_pop_df.drop_duplicates(subset=['iris2'])
    ratio=test_pop["pop_gp"]/(test_pop["pop"]+1)
    ratio_mean=np.mean(ratio)
    print(ratio_mean)
    iris_matrix_pop_df["pop_gp"]=iris_matrix_pop_df["pop_gp"]/ratio_mean

1.0192276175210808


In [254]:
matrix_df = iris_matrix_pop_df[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
matrix_df.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C
6351493,712700102,10010000,25,27,0.039164,812.583455,832.0,C
6351706,712700103,10010000,24,26,0.044157,812.583455,832.0,C
60021,14430000,10010000,21,26,0.044157,812.583455,832.0,C
6351935,712700104,10010000,25,28,0.034735,812.583455,832.0,C


In [255]:
matrix_df[matrix_df["iris2"]==debug].to_excel(writer, sheet_name=f"pop_ajustee_{ratio_mean:.3f}", index=None)
matrix_df[matrix_df["iris2"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
380294,61620000,60230000,19,24,0.056135,2440.553041,2206.0,C
380299,61630000,60230000,20,26,0.044157,2440.553041,2206.0,C
369018,61320000,60230000,8,12,0.236928,2440.553041,2206.0,C
301290,60620000,60230000,9,11,0.267135,2440.553041,2206.0,C
275796,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C


In [256]:
matrix_df["iris"] = matrix_df["iris1"].copy()
matrix_df = matrix_df.sort_values(by='iris2')
matrix_merge_df = matrix_df.merge(ps_df2, on="iris", how="left", suffixes=('', ''))
matrix_merge_df

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10280000,10010000,8,9,0.339596,812.583455,832.0,C,10280000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12890101,10010000,27,27,0.039164,812.583455,832.0,C,12890101,146499.0,810003127528,BERNOUD,FREDERIC,1.0,385966.0,5.208805,46.172593,L,146499_L_46.172593_5.208805,4.0,0.25,2.25
3,12890102,10010000,28,28,0.034735,812.583455,832.0,C,12890102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12910000,10010000,18,23,0.063292,812.583455,832.0,C,12910000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2023650000,0,0,1.000000,132.155571,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272671,2021350000,2023650000,6,8,0.382893,132.155571,100.0,C,2021350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272672,2022510000,2023650000,10,15,0.165299,132.155571,100.0,C,2022510000,503887.0,810002479136,CALENDINI,PHILIPPE,202.0,713227.0,9.366856,42.002879,L,503887_L_42.002879_9.366856,2.0,0.50,4.50
19272673,2021230000,2023650000,17,25,0.049787,132.155571,100.0,C,2021230000,1278206.0,810003447744,LARDEAUX,ALAIN,202.0,120646.0,9.403790,42.013689,L,1278206_L_42.01368918086951_9.403789965082598,1.0,1.00,3.00


In [257]:
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)].to_excel(writer, sheet_name=f"join_iris", index=None)
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)]


,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
276798,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
301311,60230000,60620000,9,11,0.267135,376.451836,310.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
370233,60230000,61320000,8,12,0.236928,545.111495,440.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
381402,60230000,61620000,19,24,0.056135,853.716514,719.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
381410,60230000,61630000,20,26,0.044157,2405.800150,2021.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75


In [258]:
matrix_merge_df["nb"] = matrix_merge_df["nb"].fillna(0)
matrix_merge_df = matrix_merge_df.sort_values(by=['iris1', "iris2"])
matrix_merge_df["nb"].nunique()

780

In [259]:
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)].to_excel(writer, sheet_name=f"nb", index=None)
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
276798,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
301311,60230000,60620000,9,11,0.267135,376.451836,310.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
370233,60230000,61320000,8,12,0.236928,545.111495,440.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
381402,60230000,61620000,19,24,0.056135,853.716514,719.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75
381410,60230000,61630000,20,26,0.044157,2405.800150,2021.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75


In [260]:
# matrix_merge_df = matrix_merge_df.sort_values(by='iris1')
matrix_merge_df["wpop"] = matrix_merge_df["accessibilite_weight"] * matrix_merge_df["pop_gp"]
matrix_merge_df["swpop"] = matrix_merge_df.groupby("iris1")["wpop"].transform("sum")
matrix_merge_df["R"] = matrix_merge_df["nb"] / (matrix_merge_df["swpop"] / 100000)
matrix_merge_df.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.0
917,10010000,10050000,21,24,0.056135,1777.796978,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.796212,19422.017054,0.0
2030,10010000,10210000,25,30,0.027324,1367.765255,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.372438,19422.017054,0.0
2594,10010000,10250000,22,28,0.034735,3735.163232,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,129.741862,19422.017054,0.0
2739,10010000,10260000,20,25,0.049787,1047.065131,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.130303,19422.017054,0.0


In [261]:
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)].to_excel(writer, sheet_name=f"R", index=None)
matrix_merge_df[(matrix_merge_df["iris"]==debug) | (matrix_merge_df["iris"]==debug)]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R
276798,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,2440.553041,2824.425157,61.95951
301311,60230000,60620000,9,11,0.267135,376.451836,310.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,100.563575,2824.425157,61.95951
370233,60230000,61320000,8,12,0.236928,545.111495,440.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,129.152045,2824.425157,61.95951
381402,60230000,61620000,19,24,0.056135,853.716514,719.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,47.923174,2824.425157,61.95951
381410,60230000,61630000,20,26,0.044157,2405.800150,2021.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,106.233322,2824.425157,61.95951


In [262]:
rgp = matrix_merge_df[matrix_merge_df["iris1"] == matrix_merge_df["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
0,10010000,10010000,C,812.583455,0.00,0.000000,19422.017054,812.583455,812.583455,832.000000
177,10020000,10020000,C,249.082431,0.00,0.000000,8927.820518,249.082431,249.082431,267.000000
265,10040101,10040101,A,1667.757499,0.00,0.000000,32545.680096,1667.757499,1667.757499,1796.951774
470,10040102,10040102,A,3626.291232,2.00,6.595003,30325.988937,3626.291232,3626.291232,3837.421806
585,10040201,10040201,A,4278.192273,7.25,23.786626,30479.311660,4278.192273,4278.192273,4393.228271
...,...,...,...,...,...,...,...,...,...,...
19272636,2023560000,2023560000,C,199.692830,0.00,0.000000,244.778550,199.692830,199.692830,148.000000
19272651,2023610000,2023610000,C,308.556888,0.00,0.000000,1887.989599,308.556888,308.556888,306.000000
19272667,2023640000,2023640000,C,51.187847,0.00,0.000000,160.421017,51.187847,51.187847,36.000000
19272670,2023650000,2023650000,C,132.155571,0.00,0.000000,1161.476181,132.155571,132.155571,100.000000


In [263]:
rgp[(rgp["iris1"]==debug) | (rgp["iris1"]==debug)].to_excel(writer, sheet_name=f"rgp", index=None)
rgp[(rgp["iris1"]==debug) | (rgp["iris1"]==debug)]

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
276798,60230000,60230000,C,2440.553041,1.75,61.95951,2824.425157,2440.553041,2440.553041,2206.0


In [264]:
apl = matrix_merge_df.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.000000,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
1,10050000,10010000,21,24,0.056135,812.583455,832.0,C,10050000,1103296.0,810002950946,PIGNARD,DOMINIQUE,1.0,79597.0,4.902674,45.998332,L,1103296_L_45.998332_4.902674,1.0,1.0,2.0,45.614180,38143.971797,5.243292,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
2,10210000,10010000,25,30,0.027324,812.583455,832.0,C,10210000,824382.0,810100903573,LEROLLE,CLAIRE,1.0,166.0,4.820810,45.993790,L,824382_L_45.99379_4.82081,1.0,1.0,4.0,22.202805,46443.947918,8.612532,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
3,10250000,10010000,22,28,0.034735,812.583455,832.0,C,10250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,28.225297,26201.721810,0.000000,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
4,10260000,10010000,20,25,0.049787,812.583455,832.0,C,10260000,505204.0,810100553667,ETIENNE,BENJAMIN,1.0,644930.0,4.928212,46.312266,L,505204_L_46.312266_4.928212,1.0,1.0,1.0,40.456148,34193.561547,2.924527,10010000,C,812.583455,0.0,0.0,19422.017054,812.583455,812.583455,832.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2022730000,2022970000,3,7,0.431711,54.715414,53.0,C,2022730000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,23.621220,349.438133,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0
19272671,2022970000,2022970000,0,0,1.000000,54.715414,53.0,C,2022970000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.715414,150.136226,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0
19272672,2022990000,2022970000,6,11,0.267135,54.715414,53.0,C,2022990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.616419,692.901504,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0
19272673,2023440000,2022970000,11,23,0.063292,54.715414,53.0,C,2023440000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.463035,625.746806,0.000000,2022970000,C,54.715414,0.0,0.0,150.136226,54.715414,54.715414,53.0


In [265]:
apl[apl["iris"]==debug].to_excel(writer, sheet_name=f"join_iris1", index=None)
apl[apl["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
1157826,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,2440.553041,2824.425157,61.95951,60230000,C,2440.553041,1.750000,61.959510,2824.425157,2440.553041,2440.553041,2206.0
1157831,60230000,60620000,9,11,0.267135,376.451836,310.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,100.563575,2824.425157,61.95951,60620000,C,376.451836,0.000000,0.000000,1775.557671,376.451836,376.451836,310.0
1157836,60230000,61320000,8,12,0.236928,545.111495,440.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,129.152045,2824.425157,61.95951,61320000,C,545.111495,0.000000,0.000000,1534.992844,545.111495,545.111495,440.0
1157841,60230000,61620000,19,24,0.056135,853.716514,719.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,47.923174,2824.425157,61.95951,61620000,C,853.716514,0.083333,4.102075,2031.492367,853.716514,853.716514,719.0
1157846,60230000,61630000,20,26,0.044157,2405.800150,2021.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,106.233322,2824.425157,61.95951,61630000,C,2405.800150,1.083333,36.916628,2934.540350,2405.800150,2405.800150,2021.0


In [266]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.0,10010000,C,812.583455,0.0,0.000000,19422.017054,812.583455,812.583455,832.0,0.000000
155,10010000,10050000,21,24,0.056135,1777.796978,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.796212,19422.017054,0.0,10050000,C,1777.796978,2.0,5.243292,38143.971797,1777.796978,1777.796978,1897.0,0.294331
386,10010000,10210000,25,30,0.027324,1367.765255,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.372438,19422.017054,0.0,10210000,C,1367.765255,4.0,8.612532,46443.947918,1367.765255,1367.765255,1498.0,0.235326
653,10010000,10250000,22,28,0.034735,3735.163232,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,129.741862,19422.017054,0.0,10250000,C,3735.163232,0.0,0.000000,26201.721810,3735.163232,3735.163232,4065.0,0.000000
808,10010000,10260000,20,25,0.049787,1047.065131,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.130303,19422.017054,0.0,10260000,C,1047.065131,1.0,2.924527,34193.561547,1047.065131,1047.065131,979.0,0.145604


In [267]:
apl[apl["iris"]==debug].to_excel(writer, sheet_name=f"ap", index=None)
apl[apl["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap
1157826,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,2440.553041,2824.425157,61.95951,60230000,C,2440.553041,1.750000,61.959510,2824.425157,2440.553041,2440.553041,2206.0,61.959510
1157831,60230000,60620000,9,11,0.267135,376.451836,310.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,100.563575,2824.425157,61.95951,60620000,C,376.451836,0.000000,0.000000,1775.557671,376.451836,376.451836,310.0,0.000000
1157836,60230000,61320000,8,12,0.236928,545.111495,440.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,129.152045,2824.425157,61.95951,61320000,C,545.111495,0.000000,0.000000,1534.992844,545.111495,545.111495,440.0,0.000000
1157841,60230000,61620000,19,24,0.056135,853.716514,719.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,47.923174,2824.425157,61.95951,61620000,C,853.716514,0.083333,4.102075,2031.492367,853.716514,853.716514,719.0,0.230269
1157846,60230000,61630000,20,26,0.044157,2405.800150,2021.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,106.233322,2824.425157,61.95951,61630000,C,2405.800150,1.083333,36.916628,2934.540350,2405.800150,2405.800150,2021.0,1.630134


In [268]:
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.000000,812.583455,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,812.583455,19422.017054,0.0,10010000,C,812.583455,0.0,0.000000,19422.017054,812.583455,812.583455,832.0,0.000000,32.678710
155,10010000,10050000,21,24,0.056135,1777.796978,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.796212,19422.017054,0.0,10050000,C,1777.796978,2.0,5.243292,38143.971797,1777.796978,1777.796978,1897.0,0.294331,32.678710
386,10010000,10210000,25,30,0.027324,1367.765255,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,37.372438,19422.017054,0.0,10210000,C,1367.765255,4.0,8.612532,46443.947918,1367.765255,1367.765255,1498.0,0.235326,32.678710
653,10010000,10250000,22,28,0.034735,3735.163232,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,129.741862,19422.017054,0.0,10250000,C,3735.163232,0.0,0.000000,26201.721810,3735.163232,3735.163232,4065.0,0.000000,32.678710
808,10010000,10260000,20,25,0.049787,1047.065131,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.130303,19422.017054,0.0,10260000,C,1047.065131,1.0,2.924527,34193.561547,1047.065131,1047.065131,979.0,0.145604,32.678710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272636,2023650000,2021350000,6,8,0.382893,451.234348,324.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,172.774422,1161.476181,0.0,2021350000,C,451.234348,0.0,0.000000,1801.097763,451.234348,451.234348,324.0,0.000000,16.284038
19272607,2023650000,2022510000,10,15,0.165299,3825.910203,3737.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,632.418703,1161.476181,0.0,2022510000,C,3825.910203,4.5,83.750308,5373.114578,3825.910203,3825.910203,3737.0,13.843833,16.284038
19272612,2023650000,2022770000,16,28,0.034735,368.164035,333.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.788273,1161.476181,0.0,2022770000,C,368.164035,0.0,0.000000,1437.844031,368.164035,368.164035,333.0,0.000000,16.284038
19272620,2023650000,2023650000,0,0,1.000000,132.155571,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,132.155571,1161.476181,0.0,2023650000,C,132.155571,0.0,0.000000,1161.476181,132.155571,132.155571,100.0,0.000000,16.284038


In [269]:
apl[apl["iris"]==debug].to_excel(writer, sheet_name=f"sum_ap", index=None)
apl[apl["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
1157826,60230000,60230000,0,0,1.000000,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,2440.553041,2824.425157,61.95951,60230000,C,2440.553041,1.750000,61.959510,2824.425157,2440.553041,2440.553041,2206.0,61.959510,63.819913
1157831,60230000,60620000,9,11,0.267135,376.451836,310.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,100.563575,2824.425157,61.95951,60620000,C,376.451836,0.000000,0.000000,1775.557671,376.451836,376.451836,310.0,0.000000,63.819913
1157836,60230000,61320000,8,12,0.236928,545.111495,440.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,129.152045,2824.425157,61.95951,61320000,C,545.111495,0.000000,0.000000,1534.992844,545.111495,545.111495,440.0,0.000000,63.819913
1157841,60230000,61620000,19,24,0.056135,853.716514,719.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,47.923174,2824.425157,61.95951,61620000,C,853.716514,0.083333,4.102075,2031.492367,853.716514,853.716514,719.0,0.230269,63.819913
1157846,60230000,61630000,20,26,0.044157,2405.800150,2021.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,106.233322,2824.425157,61.95951,61630000,C,2405.800150,1.083333,36.916628,2934.540350,2405.800150,2405.800150,2021.0,1.630134,63.819913


In [270]:
apl2 = apl[apl["iris1"] == apl["iris2"]]
apl2

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.0,812.583455,832.000000,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,812.583455,19422.017054,0.000000,10010000,C,812.583455,0.00,0.000000,19422.017054,812.583455,812.583455,832.000000,0.000000,32.678710
28748,10020000,10020000,0,0,1.0,249.082431,267.000000,C,10020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,249.082431,8927.820518,0.000000,10020000,C,249.082431,0.00,0.000000,8927.820518,249.082431,249.082431,267.000000,0.000000,37.691066
28815,10040101,10040101,0,0,1.0,1667.757499,1796.951774,A,10040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,1667.757499,32545.680096,0.000000,10040101,A,1667.757499,0.00,0.000000,32545.680096,1667.757499,1667.757499,1796.951774,0.000000,105.037626
28999,10040102,10040102,0,0,1.0,3626.291232,3837.421806,A,10040102,1294762.0,810003116513,PAGET,PHILIPPE,1.0,125.0,5.343799,45.954845,L,1294762_L_45.95484508921275_5.343798989531327,1.0,1.0,2.00,3626.291232,30325.988937,6.595003,10040102,A,3626.291232,2.00,6.595003,30325.988937,3626.291232,3626.291232,3837.421806,6.595003,105.165763
29143,10040201,10040201,0,0,1.0,4278.192273,4393.228271,A,10040201,1416059.0,810004087242,SCHODTS,PIETER JAN,1.0,88441.0,5.356510,45.961711,L,1416059_L_45.961711_5.35651,1.0,1.0,7.25,4278.192273,30479.311660,23.786626,10040201,A,4278.192273,7.25,23.786626,30479.311660,4278.192273,4278.192273,4393.228271,23.786626,112.203354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19268570,2023560000,2023560000,0,0,1.0,199.692830,148.000000,C,2023560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,199.692830,244.778550,0.000000,2023560000,C,199.692830,0.00,0.000000,244.778550,199.692830,199.692830,148.000000,0.000000,0.000000
19268441,2023610000,2023610000,0,0,1.0,308.556888,306.000000,C,2023610000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,308.556888,1887.989599,0.000000,2023610000,C,308.556888,0.00,0.000000,1887.989599,308.556888,308.556888,306.000000,0.000000,25.037451
19268523,2023640000,2023640000,0,0,1.0,51.187847,36.000000,C,2023640000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,51.187847,160.421017,0.000000,2023640000,C,51.187847,0.00,0.000000,160.421017,51.187847,51.187847,36.000000,0.000000,0.000000
19272620,2023650000,2023650000,0,0,1.0,132.155571,100.000000,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,132.155571,1161.476181,0.000000,2023650000,C,132.155571,0.00,0.000000,1161.476181,132.155571,132.155571,100.000000,0.000000,16.284038


In [271]:
apl2[apl2["iris"]==debug].to_excel(writer, sheet_name=f"apl", index=None)
writer.close()
apl2[apl2["iris"]==debug]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,inpp,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
1157826,60230000,60230000,0,0,1.0,2440.553041,2206.0,C,60230000,1398844.0,810001061463,GERSCHTEIN,JEAN LOUIS,6.0,1245439.0,7.514719,43.94556,L,1398844_L_43.94556_7.514719,6.0,0.166667,1.75,2440.553041,2824.425157,61.95951,60230000,C,2440.553041,1.75,61.95951,2824.425157,2440.553041,2440.553041,2206.0,61.95951,63.819913


In [272]:
print(year, specialite, source)
apl2["apl"].describe()
# 21-10-PA:67-57 avec ratio:68-58
# 20-10-PA:68-58 avec ratio:69-59

23 10 PA


count    48569.000000
mean        64.104830
std         35.929837
min          0.000000
25%         38.716316
50%         55.025585
75%         82.063870
max        671.098376
Name: apl, dtype: float64

In [33]:
# 21 10 PA
# count    48569.000000
# mean        66.239008
# std         36.200523
# min          0.000000
# 25%         40.432111
# 50%         56.872725
# 75%         85.083597
# max        681.212229

# 23 10 PA
# count    48569.000000
# mean        67.704332
# std         37.706173
# min          0.000000
# 25%         40.971273
# 50%         57.986870
# 75%         86.747319
# max        671.098376

# Avec prise en compte des cabinets non L
# 23 10 PA
# count    48569.000000
# mean        64.104830
# std         35.929837
# min          0.000000
# 25%         38.716316
# 50%         55.025585
# 75%         82.063870
# max        671.098376

# 24 10 PA
# count    48569.000000
# mean        62.960575
# std         36.148895
# min          0.000000
# 25%         37.356832
# 50%         53.792301
# 75%         80.853780
# max        671.109886

# 25 10 PA
# count    48569.000000
# mean        63.364476
# std         36.585777
# min          0.000000
# 25%         37.419434
# 50%         53.987877
# 75%         81.566145
# max        671.109886

In [34]:
dico = {"year":year, "specialite_id":specialite, "source":source, "time":time, "time_type":time_type, "exp":accessibilite_exp}
dico["mean"] = np.mean(apl2["apl"])
dico["std"] = np.std(apl2["apl"])
dico["q10"], dico["q25"], dico["q50"], dico["q75"], dico["q90"] = np.quantile(apl2["apl"], [0.1, 0.25, 0.5, 0.75, 0.9])
dico["min"] = np.min(apl2["apl"])
dico["max"] = np.max(apl2["apl"])
dico["date"] = datetime.datetime.now()
dico["key"] = random.randint(0, 1000000000000)
dico

{'year': 23,
 'specialite_id': 10,
 'source': 'PA',
 'time': 30,
 'time_type': 'HC',
 'exp': -0.12,
 'mean': 64.10482958844047,
 'std': 35.929467336132795,
 'q10': 27.56884058546753,
 'q25': 38.71631624897767,
 'q50': 55.02558476239584,
 'q75': 82.06386975269842,
 'q90': 115.99903198802471,
 'min': 0.0,
 'max': 671.0983762804187,
 'date': datetime.datetime(2025, 11, 6, 19, 26, 48, 672642),
 'key': 276279762794}

In [35]:
study = pd.DataFrame(dico, index=[dico["key"]])
study

,year,specialite_id,source,time,time_type,exp,mean,std,q10,q25,q50,q75,q90,min,max,date,key
276279762794,23,10,PA,30,HC,-0.12,64.10483,35.929467,27.568841,38.716316,55.025585,82.06387,115.999032,0.0,671.098376,2025-11-06 19:26:48.672642,276279762794


In [37]:
apl3 = apl2[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl3 = apl3.rename(columns={'iris1': 'iris'})
apl3.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
0,10010000,C,0.00,32.678710,0.000000,1.0,812.583455,19422.017054,0.000000,812.583455,832.000000
28748,10020000,C,0.00,37.691066,0.000000,1.0,249.082431,8927.820518,0.000000,249.082431,267.000000
28815,10040101,A,0.00,105.037626,0.000000,1.0,1667.757499,32545.680096,0.000000,1667.757499,1796.951774
28999,10040102,A,2.00,105.165763,6.595003,1.0,3626.291232,30325.988937,6.595003,3626.291232,3837.421806
29143,10040201,A,7.25,112.203354,23.786626,1.0,4278.192273,30479.311660,23.786626,4278.192273,4393.228271


In [48]:
apl3[apl3["iris"]==61070000]

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
1036622,61070000,C,0.0,117.283054,0.0,1.0,63.451557,866.963915,0.0,63.451557,65.0


In [38]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2023 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id



,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,821300000,821300000,Montjoi,82,82130,Montjoi,2023,10
1,811540000,811540000,Marnaves,81,81154,Marnaves,2023,10
2,840060000,840060000,Auribeau,84,84006,Auribeau,2023,10
3,821570000,821570000,Saint-Beauzeil,82,82157,Saint-Beauzeil,2023,10
4,840070113,840070113,Carmes-Infirmières-Thiers,84,84007,Avignon,2023,10
...,...,...,...,...,...,...,...,...
48564,940460103,940460103,Centre Vert de Maison 3,94,94046,Maisons-Alfort,2023,10
48565,940460109,940460109,Centre Vert de Maison 9,94,94046,Maisons-Alfort,2023,10
48566,940460202,940460202,Alfort 2,94,94046,Maisons-Alfort,2023,10
48567,940540105,940540105,Carrières-Sentiers,94,94054,Orly,2023,10


In [39]:
apl_final = apl3.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl_final = apl_final[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl_final["year"]=year
apl_final

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,23,10,10010000,010010000,C,0.00,32.678710,0.000000,1.0,0.000000,812.583455,19422.017054,812.583455,832.000000,L'Abergement-Clémenciat,1,01001,L'Abergement-Clémenciat
1,23,10,10020000,010020000,C,0.00,37.691066,0.000000,1.0,0.000000,249.082431,8927.820518,249.082431,267.000000,L'Abergement-de-Varey,1,01002,L'Abergement-de-Varey
2,23,10,10040101,010040101,A,0.00,105.037626,0.000000,1.0,0.000000,1667.757499,32545.680096,1667.757499,1796.951774,Les Pérouses-Triangle d'Activités,1,01004,Ambérieu-en-Bugey
3,23,10,10040102,010040102,A,2.00,105.165763,6.595003,1.0,6.595003,3626.291232,30325.988937,3626.291232,3837.421806,Longeray-Gare,1,01004,Ambérieu-en-Bugey
4,23,10,10040201,010040201,A,7.25,112.203354,23.786626,1.0,23.786626,4278.192273,30479.311660,4278.192273,4393.228271,Centre-Saint-Germain-Vareilles,1,01004,Ambérieu-en-Bugey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,23,10,2023560000,2B3560000,C,0.00,0.000000,0.000000,1.0,0.000000,199.692830,244.778550,199.692830,148.000000,Zalana,202,2B356,Zalana
48565,23,10,2023610000,2B3610000,C,0.00,25.037451,0.000000,1.0,0.000000,308.556888,1887.989599,308.556888,306.000000,Zilia,202,2B361,Zilia
48566,23,10,2023640000,2B3640000,C,0.00,0.000000,0.000000,1.0,0.000000,51.187847,160.421017,51.187847,36.000000,Zuani,202,2B364,Zuani
48567,23,10,2023650000,2B3650000,C,0.00,16.284038,0.000000,1.0,0.000000,132.155571,1161.476181,132.155571,100.000000,San-Gavino-di-Fiumorbo,202,2B365,San-Gavino-di-Fiumorbo


In [40]:
apl_final[apl_final["iris_string"]=="061070000"]

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
2499,23,10,61070000,061070000,C,0.0,117.283054,0.0,1.0,0.0,63.451557,866.963915,63.451557,65.0,La Roque-en-Provence,6,06107,La Roque-en-Provence


In [41]:
d6=apl_final[apl_final["dept"]==6]
d6

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
2095,23,10,60010000,060010000,C,0.000000,24.562258,0.000000,1.0,0.000000,97.746664,297.692517,97.746664,93.000000,Aiglun,6,06001,Aiglun
2096,23,10,60020000,060020000,C,0.000000,0.000000,0.000000,1.0,0.000000,60.236610,276.294969,60.236610,51.000000,Amirat,6,06002,Amirat
2097,23,10,60030000,060030000,C,0.000000,26.742784,0.000000,1.0,0.000000,606.375917,1311.798003,606.375917,640.000000,Andon,6,06003,Andon
2098,23,10,60040101,060040101,C,0.000000,143.700324,0.000000,1.0,0.000000,1646.292572,187948.770973,1646.292572,1763.364267,Les Trois Moulins,6,06004,Antibes
2099,23,10,60040102,060040102,A,3.000000,114.574502,2.000711,1.0,2.000711,2458.248090,149946.721270,2458.248090,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,23,10,61610105,061610105,A,8.700000,148.318824,4.312566,1.0,4.312566,3929.144712,201736.053993,3929.144712,3584.453834,Marina,6,06161,Villeneuve-Loubet
2591,23,10,61610106,061610106,A,0.000000,108.652539,0.000000,1.0,0.000000,3764.565930,146204.453272,3764.565930,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
2592,23,10,61610107,061610107,A,0.333333,118.347538,0.208379,1.0,0.208379,2691.559968,159964.686587,2691.559968,2766.654836,Plan,6,06161,Villeneuve-Loubet
2593,23,10,61620000,061620000,C,0.083333,21.715271,4.102075,1.0,4.102075,853.716514,2031.492367,853.716514,719.000000,La Brigue,6,06162,La Brigue


In [42]:
d6["apl"].describe()
# Ca change à cause des cabinets multi-dept

count    500.000000
mean      94.644163
std       49.025266
min        0.000000
25%       63.372694
50%      102.237839
75%      133.659490
max      308.069517
Name: apl, dtype: float64

In [43]:
# apl_final.to_csv("apl_france.csv", index=False)

In [44]:
# apl_final["study_key"]=dico["key"]
# study.to_sql("apl_study", config.connection_string, schema="apl", if_exists="append", index=False)
# apl_final.to_sql("apl", config.connection_string, schema="apl", if_exists="append", index=False)